# Tracking Methods: CPOL

This tutorial/demo illustrates how THUNER can be applied to [CPOL](https://www.openradar.io/research-radars/cpol), a C-band dual-polarisation research radar located at Gunn Point near Darwin, in Australia's northern Territory. 


## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

import shutil
import glob
import thuner.data as data
import thuner.option as option
import thuner.track.track as track
import thuner.visualize as visualize
import thuner.analyze as analyze
import thuner.default as default
import thuner.attribute as attribute
import thuner.parallel as parallel
import thuner.utils as utils
import thuner.config as config

In [ ]:
# Specify the local base directory for saving outputs
base_local = config.get_outputs_directory()

output_parent = base_local / "runs/cpol/geographic"
options_directory = output_parent / "options"
visualize_directory = output_parent / "visualize"

# Remove the output parent directory if it already exists
# if output_parent.exists():
    # shutil.rmtree(output_parent)

Run the cell below to get the demo data for this tutorial, if you haven't already.

In [ ]:
# Download the demo data
remote_directory = "s3://thuner-storage/THUNER_output/input_data/raw/cpol"
data.get_demo_data(base_local, remote_directory)
remote_directory = "s3://thuner-storage/THUNER_output/input_data/raw/"
remote_directory += "era5_monthly_10S_129E_14S_133E"
data.get_demo_data(base_local, remote_directory)

## Geographic Coordinates

CPOL level 1b data is provided in cartesian coordinates. We can convert this data to 
geographic coordinates on the fly by specifying default grid options. We will also save
this converted data to disk for use later.

In [ ]:
# Create the dataset options
start = "2005-11-13T14:00:00"
# Note the CPOL times are usually a few seconds off the 10 m interval, so add 30 seconds
# to ensure we capture 19:00:00
end = "2005-11-13T19:00:30" 
times_dict = {"start": start, "end": end}
cpol_options = data.aura.CPOLOptions(**times_dict, converted_options={"save": True})
# cpol_options = data.aura.CPOLOptions(**times_dict, converted_options={"load": True})
era5_dict = {"latitude_range": [-14, -10], "longitude_range": [129, 133]}
era5_pl_options = data.era5.ERA5Options(**times_dict, **era5_dict)
era5_dict.update({"data_format": "single-levels"})
era5_sl_options = data.era5.ERA5Options(**times_dict, **era5_dict)
datasets=[cpol_options, era5_pl_options, era5_sl_options]
data_options = option.data.DataOptions(datasets=datasets)
data_options.to_yaml(options_directory / "data.yml")

# Create the grid_options
grid_options = option.grid.GridOptions()
grid_options.to_yaml(options_directory / "grid.yml")

# Create the track_options
track_options = default.track(dataset_name="cpol")
# Modify the default track options to demonstrate the tracking of both convective 
# objects, and mesoscale convective systems, which are built out of convective, middle 
# and stratiform echo objects, within the same THUNER run. We will use a larger
# minimum size for the convective objects, as too many very small objects confuses the
# matching algorithm.
core = attribute.core.default_tracked()
attributes = option.attribute.Attributes(name="convective", attribute_types=[core])
track_options.levels[0].object_by_name("convective").attributes = attributes
tint_tracking = option.track.TintOptions(search_margin=5)
track_options.levels[0].object_by_name("convective").tracking = tint_tracking
mask_options = option.track.MaskOptions(save=True)
track_options.levels[0].object_by_name("convective").mask_options = mask_options
track_options.levels[0].object_by_name("convective").detection.min_area = 64
track_options.levels[0].object_by_name("convective").detection.altitudes
track_options.levels[0].object_by_name("convective").revalidate()
track_options.levels[0].revalidate()
# We will also modify the mcs tracking options to save a record of the member object ids
mcs_attributes = track_options.levels[1].object_by_name("mcs").attributes
mcs_group_attr = mcs_attributes.attribute_type_by_name("group")
membership = attribute.group.build_membership_attribute_group()
mcs_group_attr.attributes.append(membership)
mcs_group_attr.revalidate()
track_options.to_yaml(options_directory / "track.yml")

For this tutorial, we will generate figures during runtime to visualize how THUNER
is matching both convective and mcs objects.

In [ ]:
# Create the visualize_options
kwargs = {"visualize_directory": visualize_directory, "objects": ["convective", "mcs"]}
visualize_options = default.runtime(**kwargs)
visualize_options.to_yaml(options_directory / "visualize.yml")

We can now perform our tracking run; note the run will be slow as we are generating runtime figures for both convective and MCS objects, and not using parallelization. To make the run go much faster, set `visualize_options = None` and use the the parallel tracking function.

In [ ]:
times = utils.generate_times(data_options.dataset_by_name("cpol").filepaths)
args = [times, data_options, grid_options, track_options, visualize_options]
# parallel.track(*args, output_directory=output_parent)
track.track(*args, output_directory=output_parent)

Once completed, outputs are available in the `output_parent` directory. The visualization
folder will contain figures like that below, which illustrate the matching process. 
Currently THUNER supports the TINT/MINT matching approach, but the goal is to eventually 
incorporate others. Note that if viewing online, the figures below can be viewed at original scale by right clicking, save image as, and opening locally, or by right clicking, open in new tab, etc.

![Visualization of the TINT/MINT matching process.](https://raw.githubusercontent.com/THUNER-project/THUNER/refs/heads/main/gallery/cpol_convective_match_20051113.png)

Definitions of terms appearing in the above figure are provided by 
[Raut et al. (2021)](https://doi.org/10.1175/JAMC-D-20-0119.1). Note the displacement 
vector for the central orange object is large due to the object changing shape suddenly. 
Similar jumps occur when objects split and merge, and for this reason, object center displacements are ill suited to define object velocities. Instead, object velocities are calculated by smoothing the corrected local flow vectors, as discussed by [Short et al. (2023)](https://doi.org/10.1175/MWR-D-22-0146.1). Animations of all the runtime matching figures for the convective objects are provided below.

![Convective object matching.](https://raw.githubusercontent.com/THUNER-project/THUNER/refs/heads/main/gallery/cpol_convective_match_20051113.gif)

We also provide the matching figures for the MCS objects. Note there is only one MCS 
object, which is comprised of multiple disjoint convective objects; the grouping method
is described by [Short et al. (2023)](https://doi.org/10.1175/MWR-D-22-0146.1).

![MCS object matching.](https://raw.githubusercontent.com/THUNER-project/THUNER/refs/heads/main/gallery/cpol_mcs_match_20051113.gif)

Recall that when setting up the options above, we instructed THUNER to keep a record of the IDs of
each member object (convective, middle and stratiform echoes) comprising each grouped 
mcs object. Note that only the mcs and convective objects are matched between times. 

In [ ]:
filepath = output_parent / "attributes/mcs/group.csv"
columns = ["convective_ids", "middle_ids", "anvil_ids"]
print(attribute.utils.read_attribute_csv(filepath, columns=columns).to_string())

We can also perform analysis on, and visualization of, the MCS objects.

In [ ]:
analysis_options = analyze.mcs.AnalysisOptions()
analysis_options.to_yaml(options_directory / "analysis.yml")
analyze.mcs.process_velocities(output_parent)
analyze.mcs.quality_control(output_parent, analysis_options)
analyze.mcs.classify_all(output_parent, analysis_options)

In [ ]:
from thuner.visualize.attribute import AttributeHandler, AttributeVisualizeMethod, LegendArtistMethod

In [ ]:
member_objects = ["convective", "anvil"]
style = "presentation"

base_qualities = ["convective_contained", "anvil_contained", "duration"]
velocity_filepath = str(output_parent / "analysis/velocities.csv")
quality_filepath = str(output_parent / "analysis/quality.csv")
vis_func = visualize.attribute.velocity_horizontal
color, label = "tab:purple", "System Velocity"
vis_kwargs = {"color": color}
method = AttributeVisualizeMethod(function=vis_func, keyword_arguments=vis_kwargs)
leg_func = visualize.horizontal.displacement_legend_artist
leg_kwargs = {"color": color, "label": label}
legend_method = LegendArtistMethod(function=leg_func, keyword_arguments=leg_kwargs)
kwargs = {"name": "velocity", "attributes": ["u", "v"], "filepath": velocity_filepath}
kwargs.update({"method": method, "label": label, "legend_method": legend_method})
kwargs.update({"quality_filepath": quality_filepath})
kwargs.update({"quality_variables": base_qualities + ["velocity", "duration"]})
velocity_handler = AttributeHandler(**kwargs)

color, label = "tab:red", "Ambient Wind"
vis_kwargs = {"color": color}
method = AttributeVisualizeMethod(function=vis_func, keyword_arguments=vis_kwargs)
leg_func = visualize.horizontal.displacement_legend_artist
leg_kwargs = {"color": color, "label": label}
legend_method = LegendArtistMethod(function=leg_func, keyword_arguments=leg_kwargs)
kwargs = {"name": "ambient", "attributes": ["u_ambient", "v_ambient"]}
kwargs.update({"method": method, "filepath": velocity_filepath})
kwargs.update({"label": label, "legend_method": legend_method})
kwargs.update({"quality_filepath": quality_filepath})
kwargs.update({"quality_variables": base_qualities + ["duration"]})
ambient_handler = AttributeHandler(**kwargs)

color, label = "darkblue", "Ambient Shear"
vis_kwargs = {"color": color}
method = AttributeVisualizeMethod(function=vis_func, keyword_arguments=vis_kwargs)
leg_func = visualize.horizontal.displacement_legend_artist
leg_kwargs = {"color": color, "label": label}
legend_method = LegendArtistMethod(function=leg_func, keyword_arguments=leg_kwargs)
kwargs = {"name": "shear", "attributes": ["u_shear", "v_shear"]}
kwargs.update({"method": method, "filepath": velocity_filepath})
kwargs.update({"label": label, "legend_method": legend_method})
kwargs.update({"quality_filepath": quality_filepath})
kwargs.update({"quality_variables": base_qualities + ["shear", "duration"]})
shear_handler = AttributeHandler(**kwargs)

color, label = "darkgreen", "Relative System Velocity"
vis_kwargs = {"color": color}
method = AttributeVisualizeMethod(function=vis_func, keyword_arguments=vis_kwargs)
leg_func = visualize.horizontal.displacement_legend_artist
leg_kwargs = {"color": color, "label": label}
legend_method = LegendArtistMethod(function=leg_func, keyword_arguments=leg_kwargs)
kwargs = {"name": "relative", "attributes": ["u_relative", "v_relative"]}
kwargs.update({"method": method, "filepath": velocity_filepath})
kwargs.update({"label": label, "legend_method": legend_method})
kwargs.update({"quality_filepath": quality_filepath})
quality_vars = base_qualities + ["relative_velocity", "duration"]
kwargs.update({"quality_variables": quality_vars})
relative_handler = AttributeHandler(**kwargs)

vis_func = visualize.attribute.text_horizontal
vis_kwargs = {"labelled_attribute": "universal_id"}
method = AttributeVisualizeMethod(function=vis_func, keyword_arguments=vis_kwargs)
kwargs = {"name": "universal_id", "attributes": ["universal_id"], "filepath": velocity_filepath}
kwargs.update({"method": method, "label": "Object ID"})
kwargs.update({"quality_filepath": quality_filepath})
kwargs.update({"quality_variables": base_qualities})
id_handler = AttributeHandler(**kwargs)

group_filepath = str(output_parent / "attributes/mcs/group.csv")
vis_func = visualize.attribute.displacement_horizontal
color, label = "tab:blue", "Stratiform Offset"
vis_kwargs = {"color": color}
method = AttributeVisualizeMethod(function=vis_func, keyword_arguments=vis_kwargs)
leg_func = visualize.horizontal.displacement_legend_artist
leg_kwargs = {"color": color, "label": label}
legend_method = LegendArtistMethod(function=leg_func, keyword_arguments=leg_kwargs)
kwargs = {"name": "offset", "attributes": ["x_offset", "y_offset"]}
kwargs.update({"method": method, "filepath": group_filepath})
kwargs.update({"label": "Stratiform Offset"})
kwargs.update({"quality_filepath": quality_filepath})
kwargs.update({"quality_variables": base_qualities + ["offset", "duration"]})
offset_handler_convective = AttributeHandler(**kwargs)
vis_kwargs["reverse"] = True
method = AttributeVisualizeMethod(function=vis_func, keyword_arguments=vis_kwargs)
kwargs["method"] = method
offset_handler_anvil = AttributeHandler(**kwargs)

ellipse_filepath = str(output_parent / "attributes/mcs/convective/ellipse.csv")
vis_func = visualize.attribute.orientation_horizontal
method = AttributeVisualizeMethod(function=vis_func)
label = "Major Axis"
leg_func = visualize.horizontal.orientation_legend_artist
leg_kwargs = {"label": label, "style": style}
legend_method = LegendArtistMethod(function=leg_func, keyword_arguments=leg_kwargs)

kwargs = {"name": "orientation", "attributes": ["major", "orientation"]}
kwargs.update({"method": method, "filepath": ellipse_filepath, "label": label})
kwargs.update({"quality_filepath": quality_filepath, "legend_method": legend_method})
kwargs.update({"quality_variables": base_qualities + ["axis_ratio", "duration"]})
orientation_handler = AttributeHandler(**kwargs)

In [ ]:
convective_handlers = [id_handler, velocity_handler]
convective_handlers += [offset_handler_convective, orientation_handler]
anvil_handlers = [id_handler, offset_handler_anvil]
attribute_handlers = dict(zip(member_objects, [convective_handlers, anvil_handlers]))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from thuner.analyze.utils import read_options
from thuner.visualize.attribute import get_color_angle_df

In [ ]:
dataset_name = "cpol"
convective_label = "convective"

output_directory = output_parent
import xarray as xr

In [ ]:
"""Visualize mcs attributes at specified times."""
plt.close("all")
original_backend = matplotlib.get_backend()
matplotlib.use("Agg")

start_time = np.datetime64(start)
end_time = np.datetime64(end)
options = read_options(output_parent)
track_options = options["track"]
if dataset_name is None:
    try:
        object_options = track_options.levels[0].object_by_name(convective_label)
        dataset_name = object_options.dataset
    except KeyError:
        message = "Could not infer dataset used for detection. Provide manually."
        raise KeyError(message)

masks_filepath = output_directory / "masks/mcs.zarr"
masks = xr.open_dataset(masks_filepath, engine="zarr")
times = masks.time.values
times = times[(times >= start_time) & (times <= end_time)]

# Get colors
color_angle_df = get_color_angle_df(output_directory)

In [ ]:
times = utils.generate_times(data_options.dataset_by_name("cpol").filepaths)
times = list(times)

In [ ]:
record_filepath = output_directory / f"records/filepaths/{dataset_name}.csv"
filepaths = attribute.utils.read_attribute_csv(record_filepath, columns=[dataset_name])
time = times[11]

figure_name = "mcs_attributes"
kwargs = {"style": style, "attributes": ["id", "velocity", "offset"]}
kwargs["object_name"] = "mcs"
figure_options = option.visualize.HorizontalAttributeOptions(name=figure_name, **kwargs)

dt = 3600
args = [time, filepaths, masks, output_directory, figure_options]
args += [options, track_options, dataset_name, dt, color_angle_df]

In [ ]:
import thuner.detect.detect as detect
from thuner.utils import format_time
from thuner.attribute.utils import read_attribute_csv

In [ ]:
# Get object colors
plt.close("all")

keys = color_angle_df.loc[color_angle_df["time"] == time]["universal_id"].values
values = color_angle_df.loc[color_angle_df["time"] == time]["color_angle"].values
values = [visualize.visualize.mask_colormap(v / (2 * np.pi)) for v in values]
object_colors = dict(zip(keys, values))

filepath = filepaths[dataset_name].loc[time]
dataset_options = options["data"].dataset_by_name(dataset_name)

args = [time, filepath, track_options, options["grid"]]
ds, boundary_coords, simple_boundary_coords = dataset_options.convert_dataset(*args)
del boundary_coords

grid = dataset_options.grid_from_dataset(ds, "reflectivity", time)
del ds

In [ ]:
import pandas as pd

In [ ]:
processed_grid = detect.rebuild_processed_grid(grid, track_options, "mcs", 1)
del grid
mask = masks.sel(time=time)
mask = mask.load()
args = [output_directory, processed_grid, mask, simple_boundary_coords]
args += [figure_options, options["grid"]]
figure_name = figure_options.name
style = figure_options.style

In [ ]:
grid_options = options["grid"]
track_options = options["track"]
obj_name = figure_options.object_name

grid = processed_grid
time = grid.time.values

try:
    filepath = output_directory / "analysis/quality.csv"
    kwargs = {"times": [time], "columns": ["duration", "parents"]}
    object_quality = read_attribute_csv(filepath, **kwargs).loc[time]
    object_quality = object_quality.any(axis=1).to_dict()
except (FileNotFoundError, KeyError):
    object_quality = None

args = [grid, mask, grid_options, figure_options, member_objects]
args += [simple_boundary_coords]
kwargs = {"object_colors": object_colors, "mask_quality": object_quality}

with plt.style.context(visualize.visualize.styles[style]), visualize.visualize.set_style(style):
    figure_features = visualize.horizontal.grouped_mask(*args, **kwargs)
    fig, subplot_axes, colorbar_axes, legend_axes = figure_features

kwargs = {"object_name": "mcs", "time": time, "grid": grid, "mask": mask}
kwargs.update({"boundary_coordinates": simple_boundary_coords})
kwargs.update({"attribute_handlers": attribute_handlers})
kwargs.update({"member_objects": member_objects})
kwargs.update({"figure": fig, "subplot_axes": subplot_axes})
kwargs.update({"colorbar_axes": colorbar_axes, "legend_axes": legend_axes})
core_filepath = output_directory / f"attributes/{obj_name}/core.csv"
kwargs["core_filepath"] = str(core_filepath)
base_directory = output_directory / f"attributes/{obj_name}/"
filepaths_list = [str(base_directory / f"{obj}/core.csv") for obj in member_objects]
kwargs["member_core_filepaths"] = dict(zip(member_objects, filepaths_list))
grouped_figure = visualize.attribute.GroupedObjectFigure(**kwargs)

In [ ]:
from thuner.attribute.utils import read_attribute_csv

In [ ]:
legend_artists = {}
attribute_artists = {}
for i, obj in enumerate(grouped_figure.member_objects):
    attribute_artists[obj] = {}
    for handler in grouped_figure.attribute_handlers[obj]:
        # Get attribute df
        attribute_artists[obj][handler.name] = {}
        attribute_df = read_attribute_csv(handler.filepath, times=[time])
        kwargs = {"times": [time], "columns": handler.quality_variables}
        quality_df = read_attribute_csv(handler.quality_filepath, **kwargs)
        if handler.quality_method is "all":
            quality_df = quality_df.all(axis=1)
        elif handler.quality_method is "any":
            quality_df = quality_df.any(axis=1)

        try:
            id_type = "universal_id"
            object_ids = attribute_df.reset_index()[id_type].values
        except KeyError:
            id_type = "id"
            object_ids = attribute_df.reset_index()[id_type].values

        # Will also need to load in core attributes
        ax = grouped_figure.subplot_axes[i]
        core_filepath = grouped_figure.member_core_filepaths[obj]
        core_df = read_attribute_csv(core_filepath, times=[time])
        # Join the core attributes with the attribute df
        # Prepend column names with handler name if necessary
        for col in core_df.columns:
            if col in attribute_df.columns:
                core_df.rename(columns={col: f"{handler.name}_{col}"}, inplace=True)
        attribute_df = attribute_df.join(core_df)

        leg_method = handler.legend_method
        if leg_method is not None and handler.name not in legend_artists.keys():
            # Create the legend artist
            func = leg_method.function
            keyword_arguments = leg_method.keyword_arguments
            legend_artist = func(**keyword_arguments)
            legend_artists[handler.label] = legend_artist

        for obj_id in object_ids:
            # Add the attribute for the given object to the figure
            object_df = attribute_df.xs(obj_id, level=id_type, drop_level=False)
            obj_quality_df = quality_df.xs(obj_id, level=id_type, drop_level=False)
            attributes = handler.attributes
            func = handler.method.function
            kwargs = handler.method.keyword_arguments
            artist = func(ax, attributes, object_df, obj_quality_df, **kwargs)
            attribute_artists[obj][handler.name][obj_id] = artist

In [ ]:
mcs_legend_options = {"ncol": 3, "loc": "lower center"}
grouped_figure.legend_axes
scale = visualize.utils.get_extent(grid_options)[1]
handles, labels = list(legend_artists.values()), list(legend_artists.keys())
handle, handler = visualize.horizontal.mask_legend_artist()
handles += [handle]
labels += ["Object Masks"]
legend_color = visualize.visualize.figure_colors[figure_options.style]["legend"]

args = [handles, labels]

with plt.style.context(visualize.visualize.styles[style]), visualize.visualize.set_style(style):
    if scale == 1:
        legend = grouped_figure.legend_axes[0].legend(*args, **mcs_legend_options, handler_map=handler)
    elif scale == 2:
        mcs_legend_options["loc"] = "lower left"
        mcs_legend_options["bbox_to_anchor"] = (-0.0, -0.425)
        legend = grouped_figure.legend_axes[0].legend(*args, **mcs_legend_options, handler_map=handler)
    legend.get_frame().set_alpha(None)
    legend.get_frame().set_facecolor(legend_color)

In [ ]:
grouped_figure.figure

In [ ]:
with plt.style.context(visualize.visualize.styles[style]), visualize.visualize.set_style(style):

    grid_options = options["grid"]
    track_options = options["track"]
    obj_name = figure_options.object_name

    grid = processed_grid
    time = grid.time.values

    try:
        filepath = output_directory / "analysis/quality.csv"
        kwargs = {"times": [time], "columns": ["duration", "parents"]}
        object_quality = read_attribute_csv(filepath, **kwargs).loc[time]
        object_quality = object_quality.any(axis=1).to_dict()
    except (FileNotFoundError, KeyError):
        object_quality = None

    member_objects = ["convective", "anvil"]

    args = [grid, mask, grid_options, figure_options, member_objects]
    args += [simple_boundary_coords]
    kwargs = {"object_colors": object_colors, "mask_quality": object_quality}
    fig, axes, colorbar_axes, legend_axes = visualize.horizontal.grouped_mask(*args, **kwargs)

    try:
        filepath = output_directory / f"attributes/{obj_name}/core.csv"
        columns = ["latitude", "longitude"]
        core = read_attribute_csv(filepath, times=[time], columns=columns).loc[time]
        filepath = output_directory / "attributes/mcs/group.csv"
        group = read_attribute_csv(filepath, times=[time]).loc[time]
        filepath = output_directory / "analysis/velocities.csv"
        velocities = read_attribute_csv(filepath, times=[time]).loc[time]
        # filepath = output_directory / "analysis/classification.csv"
        # classification = read_attribute_csv(filepath, times=[time]).loc[time]
        filepath = output_directory / f"attributes/mcs/{convective_label}/ellipse.csv"
        ellipse = read_attribute_csv(filepath, times=[time]).loc[time]
        new_names = {"latitude": "ellipse_latitude", "longitude": "ellipse_longitude"}
        ellipse = ellipse.rename(columns=new_names)
        filepath = output_directory / "analysis/quality.csv"
        quality = read_attribute_csv(filepath, times=[time]).loc[time]
        attributes = pd.concat([core, ellipse, group, velocities, quality], axis=1)
        objs = group.reset_index()["universal_id"].values
    except KeyError:
        # If no attributes, set objs=[]
        objs = []

    for obj_id in objs:
        obj_attr = attributes.loc[obj_id]
        args = [axes, figure_options, obj_attr]
        visualize.attribute.velocity_attributes_horizontal(*args, dt=dt)
        visualize.attribute.displacement_attributes_horizontal(*args)
        visualize.attribute.ellipse_attributes(*args)
        if object_quality[obj_id]:
            visualize.attribute.text_attributes_horizontal(*args, object_quality=object_quality)

    style = figure_options.style
    scale = visualize.utils.get_extent(grid_options)[1]

    key_color = visualize.visualize.figure_colors[style]["key"]
    visualize.horizontal.vector_key(axes[0], color=key_color, dt=dt, scale=scale)
    kwargs = {"mcs_name": "mcs", "mcs_level": 1}
    convective_label, stratiform_label = visualize.attribute.get_altitude_labels(track_options, **kwargs)

    axes[0].set_title(convective_label)
    axes[1].set_title(stratiform_label)

    # Get legend proxy artists
    handles = []
    labels = []
    handle = visualize.horizontal.domain_boundary_legend_artist()
    handles += [handle]
    labels += ["Domain Boundary"]
    handle = visualize.horizontal.ellipse_legend_artist("Major Axis", figure_options.style)
    handles += [handle]
    labels += ["Major Axis"]
    attribute_names = figure_options.attributes
    for name in [attr for attr in attribute_names if attr != "id"]:
        color = visualize.attribute.colors_dispatcher[name]
        label = visualize.attribute.label_dispatcher[name]
        handle = visualize.horizontal.displacement_legend_artist(color, label)
        handles.append(handle)
        labels.append(label)

    handle, handler = visualize.horizontal.mask_legend_artist()
    handles += [handle]
    labels += ["Object Masks"]
    legend_color = visualize.visualize.figure_colors[figure_options.style]["legend"]
    handles, labels = handles[::-1], labels[::-1]

    mcs_legend_options = {"ncol": 3, "loc": "lower center"}

    args = [handles, labels]
    leg_ax = legend_axes[0]
    if scale == 1:
        legend = leg_ax.legend(*args, **mcs_legend_options, handler_map=handler)
    elif scale == 2:
        mcs_legend_options["loc"] = "lower left"
        mcs_legend_options["bbox_to_anchor"] = (-0.0, -0.425)
        legend = leg_ax.legend(*args, **mcs_legend_options, handler_map=handler)
    legend.get_frame().set_alpha(None)
    legend.get_frame().set_facecolor(legend_color)

    # Remove mask and processed_grid from memory after generating the figure
    del mask, processed_grid
    filename = f"{format_time(time)}.png"
    filepath = output_directory / f"visualize/{figure_name}/{filename}"
    filepath.parent.mkdir(parents=True, exist_ok=True)
    plt.show()
    fig.savefig(filepath, bbox_inches="tight")
    visualize.utils.reduce_color_depth(filepath)

In [ ]:
figure_name = "mcs_attributes"
kwargs = {"style": "presentation", "attributes": ["id", "velocity", "offset"]}
figure_options = option.visualize.HorizontalAttributeOptions(name=figure_name, **kwargs)

args = [output_parent, start, end, figure_options]
args_dict = {"parallel_figure": True, "by_date": False, "num_processes": 4}
visualize.attribute.mcs_series(*args, **args_dict)

## Pre-Converted Data

We can also perform THUNER tracking runs on general datasets, we just need to ensure 
they are pre-converted into a format recognized by THUNER, i.e. gridded data files readable by 
``xarray.open_dataset``, with variables named according to [CF-conventions](https://cfconventions.org/).
To illustrate, we will use the converted CPOL files that were generated by the code in the
previous section. We first modify the options used for the geographic coordinates above. Re-run
the relevant cells above again if necessary. If you get a pydantic error, restart the notebook.

In [ ]:
output_parent = base_local / "runs/cpol/pre_converted"
options_directory = output_parent / "options"
options_directory.mkdir(parents=True, exist_ok=True)

if output_parent.exists():
    shutil.rmtree(output_parent)

# Get the pre-converted filepaths
base_filepath = base_local / "input_data/converted/cpol/cpol_level_1b/v2020/gridded/"
base_filepath = base_filepath / "grid_150km_2500m/2005/20051113"
filepaths = glob.glob(str(base_filepath / "*.nc"))
filepaths = sorted(filepaths)

# Create the data options. 
kwargs = {"name": "cpol", "fields": ["reflectivity"], "filepaths": filepaths}
cpol_options = utils.BaseDatasetOptions(**times_dict, **kwargs)
datasets=[cpol_options, era5_pl_options, era5_sl_options]
data_options = option.data.DataOptions(datasets=datasets)
data_options.to_yaml(options_directory / "data.yml")

# Save other options
grid_options.to_yaml(options_directory / "grid.yml")
track_options.to_yaml(options_directory / "track.yml")

# Switch off the runtime figures
visualize_options = None

In [ ]:
times = utils.generate_times(data_options.dataset_by_name("cpol").filepaths)
args = [times, data_options, grid_options, track_options, visualize_options]
kwargs = {"output_directory": output_parent, "dataset_name": "cpol"}
parallel.track(*args, **kwargs, debug_mode=True)

In [ ]:
analysis_options = analyze.mcs.AnalysisOptions()
analysis_options.to_yaml(options_directory / "analysis.yml")
analyze.mcs.process_velocities(output_parent)
analyze.mcs.quality_control(output_parent, analysis_options)
analyze.mcs.classify_all(output_parent, analysis_options)

In [ ]:
figure_name = "mcs_attributes"
kwargs = {"style": "presentation", "attributes": ["id", "velocity", "offset"]}
figure_options = option.visualize.HorizontalAttributeOptions(name=figure_name, **kwargs)

args = [output_parent, start, end, figure_options]
args_dict = {"parallel_figure": True, "by_date": False, "num_processes": 4}
visualize.attribute.mcs_series(*args, **args_dict)

Note we can achieve the same result in this case by modifying `converted_options={"save": True}` to `converted_options={"load": True}` in the [Geographic Coordinates](#geographic-coordinates) section,and rerunning the cells.

## Cartesian Coordinates

Because the CPOL radar domains are small (150 km radii), it is reasonable to perform 
tracking in Cartesian coordinates. This should make the run faster as we are no longer 
performing regridding on the fly. We will also switch off the runtime figure generation.

In [ ]:
output_parent = base_local / "runs/cpol/cartesian"
options_directory = output_parent / "options"
options_directory.mkdir(parents=True, exist_ok=True)

if output_parent.exists():
    shutil.rmtree(output_parent)

# Recreate the original cpol dataset options
cpol_options = data.aura.CPOLOptions(**times_dict)
datasets = [cpol_options, era5_pl_options, era5_sl_options]
data_options = option.data.DataOptions(datasets=datasets)
data_options.to_yaml(options_directory / "data.yml")

# Create the grid_options
grid_options = option.grid.GridOptions(name="cartesian", regrid=False)
grid_options.to_yaml(options_directory / "grid.yml")

# Save the same track options from earlier
track_options.to_yaml(options_directory / "track.yml")
visualize_options = None

In [ ]:
times = utils.generate_times(data_options.dataset_by_name("cpol").filepaths)
args = [times, data_options, grid_options, track_options, visualize_options]
kwargs = {"output_directory": output_parent, "dataset_name": "cpol"}
parallel.track(*args, **kwargs)

In [ ]:
analysis_options = analyze.mcs.AnalysisOptions()
analysis_options.to_yaml(options_directory / "analysis.yml")
analyze.mcs.process_velocities(output_parent)
analyze.mcs.quality_control(output_parent, analysis_options)
analyze.mcs.classify_all(output_parent, analysis_options)

In [ ]:
figure_name = "mcs_attributes"
kwargs = {"style": "presentation", "attributes": ["id", "velocity", "offset"]}
figure_options = option.visualize.HorizontalAttributeOptions(name=figure_name, **kwargs)

args = [output_parent, start, end, figure_options]
args_dict = {"parallel_figure": True, "by_date": False, "num_processes": 4}
visualize.attribute.mcs_series(*args, **args_dict)